# Firefly Algorithm: Intuition and Applications to Rastrigin & Knapsack

**Objectives:**
- Explain the core intuition behind the Firefly Algorithm (FA)
- Visualize how FA explores the search space in 2D
- Demonstrate FA on the continuous Rastrigin problem
- Demonstrate FA on the discrete 0–1 Knapsack problem
- Compare convergence behavior across problem types

---

## 1. Intuition: What is the Firefly Algorithm?

The **Firefly Algorithm** is a nature-inspired metaheuristic optimization algorithm based on the flashing behavior of fireflies.

**Core concepts:**
- A population of **fireflies** represents candidate solutions
- Each firefly has a **brightness** proportional to its objective value (fitness)
- Fireflies are **attracted** to brighter fireflies and move toward them
- **Randomness** allows exploration of new regions

**Key equations:**

1. **Attractiveness** decreases with distance:
   $$\beta(r) = \beta_0 \cdot e^{-\gamma r^2}$$
   - $\beta_0$: base attractiveness (typically 1.0)
   - $\gamma$: light absorption coefficient (controls how fast attraction decays)
   - $r$: Euclidean distance between two fireflies

2. **Position update** when firefly $i$ is attracted to brighter firefly $j$:
   $$\mathbf{x}_i \leftarrow \mathbf{x}_i + \beta(r) (\mathbf{x}_j - \mathbf{x}_i) + \alpha \cdot \mathbf{\epsilon}$$
   - First term: current position
   - Second term: attraction toward brighter firefly
   - Third term: random walk ($\alpha$ controls randomness, $\mathbf{\epsilon}$ is random noise)

**Parameter roles:**
- $\alpha$: controls exploration (higher = more random)
- $\beta_0$: controls attraction strength
- $\gamma$: controls interaction range (higher = more local search)

---

## 2. Visual Demo: FA in 2D

Let's build a simple 2D Firefly Algorithm from scratch to visualize the movement pattern.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Define a 2D test function: shifted Rastrigin (for visual clarity)
def objective_2d(x, y):
    """2D Rastrigin-like function with optimum at (0, 0)"""
    return 20 + (x**2 - 10*np.cos(2*np.pi*x)) + (y**2 - 10*np.cos(2*np.pi*y))

# Simple FA implementation for 2D visualization
class SimpleFA2D:
    def __init__(self, n_fireflies=15, alpha=0.2, beta0=1.0, gamma=1.0, bounds=(-5, 5)):
        self.n = n_fireflies
        self.alpha = alpha
        self.beta0 = beta0
        self.gamma = gamma
        self.bounds = bounds
        
    def initialize(self):
        """Randomly initialize firefly positions"""
        low, high = self.bounds
        self.positions = np.random.uniform(low, high, (self.n, 2))
        
    def brightness(self, pos):
        """Calculate brightness (inverse of objective)"""
        return -objective_2d(pos[0], pos[1])  # Minimize objective
    
    def distance(self, pos_i, pos_j):
        """Euclidean distance"""
        return np.linalg.norm(pos_i - pos_j)
    
    def attractiveness(self, r):
        """Beta decreases with distance"""
        return self.beta0 * np.exp(-self.gamma * r**2)
    
    def run(self, iterations=30):
        """Run FA and store trajectory"""
        self.initialize()
        trajectory = [self.positions.copy()]
        
        for _ in range(iterations):
            # Calculate brightness for all fireflies
            brightness = np.array([self.brightness(pos) for pos in self.positions])
            
            # Move each firefly toward brighter ones
            new_positions = self.positions.copy()
            for i in range(self.n):
                for j in range(self.n):
                    if brightness[j] > brightness[i]:  # j is brighter than i
                        r = self.distance(self.positions[i], self.positions[j])
                        beta = self.attractiveness(r)
                        # Move i toward j with randomness
                        new_positions[i] += beta * (self.positions[j] - self.positions[i])
                        new_positions[i] += self.alpha * np.random.uniform(-1, 1, 2)
                
                # Keep within bounds
                new_positions[i] = np.clip(new_positions[i], *self.bounds)
            
            self.positions = new_positions
            trajectory.append(self.positions.copy())
        
        return trajectory

print("✓ Simple 2D FA implementation ready")

In [ ]:
# Run the 2D FA
fa_2d = SimpleFA2D(n_fireflies=15, alpha=0.2, beta0=1.0, gamma=1.0, bounds=(-5, 5))
trajectory = fa_2d.run(iterations=30)

print(f"Collected trajectory for {len(trajectory)} iterations")
print(f"Number of fireflies: {fa_2d.n}")

In [ ]:
# Visualize: snapshots at key iterations
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Create contour background
x = np.linspace(-5, 5, 200)
y = np.linspace(-5, 5, 200)
X, Y = np.meshgrid(x, y)
Z = objective_2d(X, Y)

# Snapshots at iterations: 0, 5, 10, 15, 20, 30
snapshot_iters = [0, 5, 10, 15, 20, 29]

for idx, iter_num in enumerate(snapshot_iters):
    ax = axes[idx]
    
    # Plot contour
    contour = ax.contourf(X, Y, Z, levels=20, cmap='viridis', alpha=0.7)
    
    # Plot fireflies at this iteration
    positions = trajectory[iter_num]
    ax.scatter(positions[:, 0], positions[:, 1], c='red', s=100, 
              edgecolors='white', linewidth=2, marker='o', zorder=5, alpha=0.9)
    
    # Mark global optimum
    ax.scatter(0, 0, c='gold', s=300, marker='*', 
              edgecolors='black', linewidth=2, zorder=10)
    
    ax.set_xlim(-5, 5)
    ax.set_ylim(-5, 5)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title(f'Iteration {iter_num}', fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🔥 Notice how fireflies cluster around good regions and converge toward the optimum!")

In [ ]:
# Alternative: show trails (trajectory lines)
fig, ax = plt.subplots(figsize=(10, 8))

# Plot contour
contour = ax.contourf(X, Y, Z, levels=25, cmap='viridis', alpha=0.6)
plt.colorbar(contour, ax=ax, label='Objective Value')

# Plot trajectory for each firefly
for firefly_idx in range(fa_2d.n):
    path = np.array([traj[firefly_idx] for traj in trajectory])
    ax.plot(path[:, 0], path[:, 1], 'o-', alpha=0.4, linewidth=1, markersize=3)
    
    # Mark start (green) and end (red)
    ax.scatter(path[0, 0], path[0, 1], c='lime', s=80, marker='o', 
              edgecolors='black', linewidth=1.5, zorder=5, alpha=0.8)
    ax.scatter(path[-1, 0], path[-1, 1], c='red', s=120, marker='o', 
              edgecolors='white', linewidth=2, zorder=5)

# Mark global optimum
ax.scatter(0, 0, c='gold', s=400, marker='*', 
          edgecolors='black', linewidth=3, zorder=10, label='Global Optimum')

ax.set_xlabel('X', fontsize=12)
ax.set_ylabel('Y', fontsize=12)
ax.set_title('Firefly Trajectories on 2D Landscape', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Application to Rastrigin Function (High-Dimensional Continuous)

The **Rastrigin function** is a standard benchmark for optimization:
- **Highly multimodal** (many local optima)
- **Non-separable** in higher dimensions
- Global optimum at $\mathbf{x} = \mathbf{0}$ with $f(\mathbf{0}) = 0$

Formula: $f(\mathbf{x}) = An + \sum_{i=1}^{n}[x_i^2 - A\cos(2\pi x_i)]$ where $A=10$

We'll use the **existing FA implementation** from this repository.

In [ ]:
import sys
import os

# Add parent directory to path
sys.path.append(os.path.dirname(os.path.abspath('')))

from src.problems.continuous.rastrigin import RastriginProblem
from src.swarm.fa import FireflyContinuousOptimizer

print("✓ Imported project's FA implementation")

In [ ]:
# Setup Rastrigin problem in 10 dimensions
dim = 10
problem = RastriginProblem(dim=dim)

# Configure FA
fa_rastrigin = FireflyContinuousOptimizer(
    problem=problem,
    n_fireflies=30,
    alpha=0.2,
    beta0=1.0,
    gamma=1.0,
    seed=42
)

# Run optimization
print(f"Running FA on {dim}D Rastrigin...")
best_solution, best_fitness, history, _ = fa_rastrigin.run(max_iter=100)

print(f"\n✓ Optimization complete!")
print(f"Best fitness found: {best_fitness:.6f}")
print(f"Global optimum: 0.0")
print(f"Initial fitness: {history[0]:.6f}")
print(f"Improvement: {history[0] - best_fitness:.6f}")

In [ ]:
# Plot convergence curve for Rastrigin
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(history, linewidth=2.5, color='#2E86AB', label='FA on Rastrigin')
ax.axhline(y=0, color='red', linestyle='--', linewidth=2, label='Global Optimum')

ax.set_xlabel('Iteration', fontsize=12)
ax.set_ylabel('Best Fitness', fontsize=12)
ax.set_title(f'FA Convergence on {dim}D Rastrigin Function', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"📊 Observations:")
print(f"   - FA quickly improves in early iterations")
print(f"   - Convergence slows as it approaches local/global optima")
print(f"   - In high dimensions, reaching exact global optimum (0.0) is very difficult")

## 4. Application to 0–1 Knapsack Problem (Discrete/Combinatorial)

The **0–1 Knapsack** is a classic combinatorial optimization problem:
- Given $n$ items, each with value $v_i$ and weight $w_i$
- Select a subset to maximize total value
- Subject to: total weight ≤ capacity $C$

**Challenge:** FA was designed for continuous spaces, so the implementation uses:
- **Binary encoding** (0 = don't take item, 1 = take item)
- **Repair mechanisms** to ensure feasibility (total weight ≤ capacity)
- **Discrete movement** via probabilistic bit flips

We'll use the existing FA implementation for Knapsack.

In [ ]:
from src.problems.discrete.knapsack import KnapsackProblem
from src.swarm.fa import FireflyKnapsackOptimizer

print("✓ Imported Knapsack problem and FA optimizer")

In [ ]:
# Create a Knapsack instance
np.random.seed(123)
n_items = 30
values = np.random.randint(10, 100, n_items)
weights = np.random.randint(5, 50, n_items)
capacity = int(0.5 * np.sum(weights))  # 50% of total weight

problem_knapsack = KnapsackProblem(values, weights, capacity)

print(f"Knapsack Problem:")
print(f"  Number of items: {n_items}")
print(f"  Capacity: {capacity}")
print(f"  Total weight if all taken: {np.sum(weights)}")
print(f"  Total value if all taken: {np.sum(values)}")

In [ ]:
# Configure FA for Knapsack
fa_knapsack = FireflyKnapsackOptimizer(
    problem=problem_knapsack,
    n_fireflies=25,
    alpha_flip=0.2,
    max_flips_per_move=3,
    seed=42
)

# Run optimization
print(f"\nRunning FA on Knapsack...")
best_solution, best_fitness, history, _ = fa_knapsack.run(max_iter=100)

# FA returns negative fitness (for minimization), negate to get actual value
best_value = -best_fitness
best_weight = np.sum(best_solution * weights)

print(f"\n✓ Optimization complete!")
print(f"Best total value: {best_value:.0f}")
print(f"Best total weight: {best_weight} / {capacity}")
print(f"Feasible: {best_weight <= capacity}")
print(f"Items selected: {np.sum(best_solution)} / {n_items}")
print(f"Initial value: {-history[0]:.0f}")
print(f"Improvement: {best_value - (-history[0]):.0f}")

In [ ]:
# Plot convergence curve for Knapsack (negate for actual values)
fig, ax = plt.subplots(figsize=(10, 6))

actual_values = [-h for h in history]  # Convert to maximization (total value)
ax.plot(actual_values, linewidth=2.5, color='#A23B72', label='FA on Knapsack')

ax.set_xlabel('Iteration', fontsize=12)
ax.set_ylabel('Total Value (Profit)', fontsize=12)
ax.set_title(f'FA Convergence on 0–1 Knapsack ({n_items} items)', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"📊 Observations:")
print(f"   - Rapid improvement in early iterations (greedy-like behavior)")
print(f"   - Convergence plateaus as feasible space is explored")
print(f"   - Discrete nature causes step-like improvements")

In [ ]:
# Visualize the solution: which items were selected?
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left plot: value and weight of all items
ax1.scatter(weights, values, c='lightgray', s=100, alpha=0.6, label='Not selected')
selected_mask = best_solution == 1
ax1.scatter(weights[selected_mask], values[selected_mask], 
           c='#A23B72', s=150, marker='s', edgecolors='black', linewidth=2,
           label='Selected', zorder=5)

ax1.set_xlabel('Weight', fontsize=12)
ax1.set_ylabel('Value', fontsize=12)
ax1.set_title('Item Selection in Value-Weight Space', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Right plot: bar chart of selected items
selected_indices = np.where(selected_mask)[0]
selected_vals = values[selected_mask]
selected_wts = weights[selected_mask]

x_pos = np.arange(len(selected_indices))
bars = ax2.bar(x_pos, selected_vals, color='#A23B72', alpha=0.7, edgecolor='black', linewidth=1.5)

# Overlay weights as line
ax2_twin = ax2.twinx()
ax2_twin.plot(x_pos, selected_wts, 'o-', color='red', linewidth=2, markersize=8, label='Weight')

ax2.set_xlabel('Selected Item Index', fontsize=12)
ax2.set_ylabel('Value', fontsize=12, color='#A23B72')
ax2_twin.set_ylabel('Weight', fontsize=12, color='red')
ax2.set_title(f'Selected Items ({len(selected_indices)} total)', fontsize=13, fontweight='bold')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(selected_indices, rotation=45)
ax2_twin.legend(loc='upper right', fontsize=9)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 5. Comparison: Rastrigin vs Knapsack

Let's visually compare the convergence behavior on both problems.

In [ ]:
# Side-by-side comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Rastrigin
ax1.plot(history, linewidth=2.5, color='#2E86AB')
ax1.axhline(y=0, color='red', linestyle='--', linewidth=2, alpha=0.7)
ax1.set_xlabel('Iteration', fontsize=12)
ax1.set_ylabel('Best Fitness (minimize)', fontsize=12)
ax1.set_title(f'Rastrigin ({dim}D) - Continuous', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Knapsack
actual_values_knapsack = [-h for h in history]
ax2.plot(actual_values_knapsack, linewidth=2.5, color='#A23B72')
ax2.set_xlabel('Iteration', fontsize=12)
ax2.set_ylabel('Total Value (maximize)', fontsize=12)
ax2.set_title(f'Knapsack ({n_items} items) - Discrete', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Key differences:")
print("   - Rastrigin: smooth continuous convergence, harder to reach global optimum")
print("   - Knapsack: step-like discrete improvements, fast early gains, then plateau")

## 6. Summary and Key Talking Points

**What is the Firefly Algorithm?**
- Nature-inspired metaheuristic based on firefly attraction
- Population-based: maintains multiple candidate solutions
- Movement driven by attractiveness (brightness) and randomness
- Parameters $\alpha$, $\beta_0$, $\gamma$ control exploration vs exploitation

**2D Visualization Insights:**
- Fireflies cluster around promising regions
- Random component prevents premature convergence
- Attraction mechanism balances local and global search

**Rastrigin (Continuous, Multimodal):**
- FA successfully improves the solution over iterations
- Highly multimodal landscape makes finding exact global optimum difficult
- Smooth convergence curve typical of continuous optimization
- Performance depends on parameter tuning (especially $\gamma$ for locality)

**Knapsack (Discrete, Combinatorial):**
- FA adapted to discrete space via binary encoding + repair
- Fast early improvements due to greedy repair mechanism
- Plateaus as feasible high-value solutions are found
- Step-like convergence reflects discrete nature of problem

**Strengths of FA:**
- Intuitive metaphor (easy to understand and explain)
- Flexible: can be adapted to continuous and discrete problems
- Population-based: explores multiple regions simultaneously
- Few parameters compared to some other metaheuristics

**Limitations:**
- No guarantee of finding global optimum (like all metaheuristics)
- Performance sensitive to parameter settings
- Can be slow on very high-dimensional problems
- Requires problem-specific adaptations for discrete spaces

**Overall:**
FA is a powerful, general-purpose optimization tool. It works well on moderately complex problems and provides good solutions when exact methods are infeasible. The key is understanding the problem structure and tuning parameters appropriately.

---

**End of Notebook** 🔥